In [1]:
#hydrogen price from electrolysis
#Adapted from International Energy Agency G20 Hydrogen report
#2017 USD costs
#discount rate

In [2]:
#interest rate used in discounted cash flow (DCF) analysis to determine the present value of future cash flows
#In DCF, the discount rate expresses the time value of money
#the discount rate refers to the interest rate used to determine the present value.
#In either case, the net present value of all cash flows should be positive to proceed with the investment or the project.

In [3]:
#global variables
LHV = 33 #kWe kgH2 (LHV)
discountRate = 8 #% - roughly the intrest rate on loans for CAPEX
inflation = 2 #% - to adjust pricing of electricity and hydrogen

In [4]:
#Electrolyser Variable

#Usage
hoursPerDay = 24 #hours
daysPerYear = 340 #days
averageNominalPower = 80#hours
use = hoursPerDay * daysPerYear * averageNominalPower/100 #effective hours per year

#Electrolyser
efficiency = 69 #% equivalent to 1.78 V
efficiencyLoss = 1.5 #%efficiency loss per year equivalent to 2.25 muV operation-1 
lifeHours = 80000 #h of useable hours
lifeYears = lifeHours/(hoursPerDay * daysPerYear) #number of years electrolyser is in operation
powerRequired = LHV/(efficiency/100)# kW H2

#CAPEX
equipmentCost = 250 #GBP kW 250-1000 euro
CAPEX = equipmentCost*powerRequired/lifeHours #price per Kg H2 produced overlifetime

#FINANCING CAPEX
FINANCING = 0

#ENERGY OPEX
elecPrice = 0.03 #mean electricity price in GBP / kWh
energyOPEX = powerRequired*elecPrice #GBP per kg H2

#MAINTAINANCE OPEX
maintanceCost = 1.5 #1.5 % of CAPEX per year
maintOPEX = maintanceCost/100*equipmentCost*lifeYears/lifeHours

OPEX = maintOPEX + energyOPEX

total = FINANCING + CAPEX + OPEX #GBP kW-1

In [5]:
#compression costs

In [6]:
#Excluding distribution costs / local generation / central generation
#make a link to a polarisation curve and a price minimisation stragety
#can we make wind baseload like with hydrogen production - how much hydrogen would we make compared to energy use?
#what would the price of hydrogen be?
#price of steam methane reforming??
#financing cost

In [7]:
#CH4 and CCUS
capex: 1360 usd kWH2 
efficiency: 69%
opex: 3% of CAPEX
CO2 capture rate: 90%
emission factor: 1.0 kgCO2/kgH2
CO2 price:100 USD / tCO2 (2050 -160 logn term)
    
gasPrice : 8 USD M BTU 

CO2 transport and storage costs: 20 USD / tonne
water costs not considered



SyntaxError: invalid syntax (<ipython-input-7-73d1b147958d>, line 2)

In [53]:
#bokeh interactive plotting
import bokeh
import bokeh.plotting as plt
from bokeh.models import ColumnDataSource, LinearAxis, Range1d
from bokeh.io import show, output_file
from bokeh.plotting import figure
plt.reset_output()
plt.output_notebook()
import math
from bokeh.models import Span, Label

Loading BokehJS ...

In [9]:
output_file("static.html")

technology = ['PEME', 'Alkaline', 'SMR', 'ATR']
costs = ['Electricity', 'Gas', 'Other OpEx', 'CapEx', 'Finance']
colors = ['#005596', '#00AEEF', '#791D7E', '#6DB33F',  '#EE3224', '#FFC425'][0:5]

data = {'technology' : technology,
        'Electricity'   : [2, 1, 4, 3],
        'Gas'   : [5, 3, 4, 2],
        'Other OpEx'   : [3, 2, 4, 4],
        'CapEx'   : [3, 2, 4, 4],
        'Finance'   : [3, 2, 4, 4],       
       }

p = figure(x_range=technology, plot_height=250, title="Price of 8 MPa Hydrogen",
           toolbar_location=None, tools="hover", tooltips="$name @technology: @$name")

p.vbar_stack(costs, x='technology', width=0.9, source=data, color=colors
            )

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

C:\tools\Anaconda3\lib\site-packages\bokeh\models\plots.py:767: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
C:\tools\Anaconda3\lib\site-packages\bokeh\models\plots.py:767: UserWarning: 
You are attempting to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


404

In [69]:
output_file("dynamic.html")

technology = ['PEME', 'Alkaline', 'SMR', 'ATR']
costs = ['Electricity', 'Gas', 'Other OpEx', 'CapEx', 'Finance']
colors = ['#005596', '#00AEEF', '#791D7E', '#6DB33F',  '#EE3224', '#FFC425'][0:5]

data = {'technology' : technology,
        'Electricity'   : [2, 1, 4, 3],
        'Gas'   : [5, 3, 4, 2],
        'Other OpEx'   : [3, 2, 4, 4],
        'CapEx'   : [3, 2, 4, 4],
        'Finance'   : [3, 2, 4, 4],       
       }

height=int(400)
width = int(height*1.618)
p = figure(x_range=technology, plot_height=height, plot_width=width,
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

p.vbar_stack(costs, x='technology', width=0.8, source=data, color=colors, legend_label=costs)


pumpPrice = Span(location=2, dimension='width', line_color='green', line_dash='dashed', line_width=3)
pumpPriceLabel = Label(x=4.5, y=1, text='Pump Price', render_mode='css', border_line_color=None)

p.add_layout(pumpPrice)
p.add_layout(pumpPriceLabel)


p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = 'Black'

p.y_range.end = 25
p.y_range.start = 0
p.legend.border_line_color = None
p.legend.location = 'top_center'
p.legend.orientation = 'horizontal'

show(p)